In [1]:
import pandas as pd
import torch
import torch.optim as optim
import os

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn

In [2]:
def FEN_to_bit_vector(fen):
    # Converting the Peices to a bit vector
    piece_layer = {
        'P': 0,
        'N': 1,
        'B': 2,
        'R': 3,
        'Q': 4,
        'K': 5,
        'p': 6,
        'n': 7,
        'b': 8,
        'r': 9,
        'q': 10,
        'k': 11
    }
    
    fen = fen.split(' ')
    piece_vector = torch.zeros(12, 8, 8)
    pieces = fen[0]
    rows = pieces.split('/')
    for i, row in enumerate(rows):
        j = 0
        for c in row:
            if c.isdigit():
                j += int(c)
            else:
                piece_vector[piece_layer[c], i, j] = 1
                j += 1
                
    # Converting the castling rights to a bit vector
    castling_vector = torch.zeros(4)
    castling = fen[2]
    for c in castling:
        if c == 'K':
            castling_vector[0] = 1
        if c == 'Q':
            castling_vector[1] = 1
        if c == 'k':
            castling_vector[2] = 1
        if c == 'q':
            castling_vector[3] = 1
            
    # Converting the en passant square to a bit vector
    en_passant_vector = torch.zeros(8)
    en_passant = fen[3]
    if en_passant != '-':
        en_passant = ord(en_passant[0]) - 97
        en_passant_vector[en_passant] = 1
        
    # Getting the current player
    curr_player_vector = torch.zeros(1)
    curr_player = fen[1]
    
    if curr_player == 'w':
        curr_player_vector[0] = 1
        
    # Append all the bit vectors
    bit_vector = torch.cat((piece_vector.view(-1), castling_vector, en_passant_vector, curr_player_vector))
    
    return bit_vector
    

def eval_to_int(eval):
    try:
        res = int(eval)
    except ValueError:
        res = 10000 if eval[1] == '+' else -10000
        
    return torch.tensor(res / 100, dtype=torch.float32)

class ChessDataset(Dataset):
    def __init__(self, csv):
        self.csv = csv
        
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, idx):
        x = FEN_to_bit_vector(self.csv.iloc[idx]['FEN'])
        y = eval_to_int(self.csv.iloc[idx]['Evaluation'])
        return x, y

In [3]:
class ChessModel(nn.Module):
    def __init__(self):
        super(ChessModel, self).__init__()
        self.linear1 = nn.Linear(12 * 8 * 8 + 4 + 8 + 1, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 128)
        self.linear4 = nn.Linear(128, 64)
        self.linear5 = nn.Linear(64, 32)
        self.linear6 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        x = torch.relu(self.linear4(x))
        x = torch.relu(self.linear5(x))
        x = self.linear6(x)
        return x

In [4]:
# Load the dataset
df = pd.read_csv("data/tactic_evals.csv")

row = df.iloc[10]

print(row['Evaluation'])

+667


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = ChessDataset(df)

train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    
model = ChessModel().to(device)
model = nn.DataParallel(model)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
        

d:\NUST\Semester 3\Artificial Intelligence\End Semester Project\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error importing huggingface_hub.hf_api: No module named 'yaml'


In [42]:
dataset[10][0].shape

torch.Size([781])

In [ ]:
losses = []

# Train the model
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader), 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, loss: {running_loss / len(train_loader)}")
    losses.append(running_loss / len(train_loader))

 11%|█         | 17380/156051 [07:29<2:28:43, 15.54it/s] 

In [ ]:
model.eval()
running_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(tqdm(val_loader), 0):
        inputs, labels = data
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        running_loss += loss.item()
print(f"Validation loss: {running_loss / len(val_loader)}")

torch.save(model.state_dict(), "model.pth")
    

100%|██████████| 8214/8214 [01:37<00:00, 84.51it/s] 

Validation loss: 445.23081884484003
